In [2]:
# -*- coding: utf-8 -*-
import numpy  as np
import matplotlib.pyplot as plt
import pandas as pd

In [59]:
C_L = 1.4  # 最大揚力係数
C_ROOT = 2.13 * 1000  # rootのchord長[mm]
C_TIP = 1.07 * 1000  # tipのroot長[mm]
ALPHA_F = np.deg2rad(14.5)# [rad]
W = 1500 * 9.8  # 自重[N]
N_Z = 6  # 最大荷重倍数
DELTA_Y = 25  # [mm]
HARF_SPAN = 5000  # [mm]
Y_REP_FOR_C = 5000 * np.array([0, 0.2, 0.4, 0.6, 0.8, 0.9, 0.95, 0.975, 1.0])  # [mm]
C_LA_REP = np.array([0.835, 1.021, 1.095, 1.089, 0.993,
                     0.833, 0.662, 0.548, 0])  # [no dim]
C_LB_REP = np.array([0.049, 0.044, 0.005, -0.033, -
                     0.062, -0.067, -0.056, -0.043, 0])  # [no dim]
C_D_REP = np.array([0.1679, 0.1303, 0.1105, 0.1065, 0.1163,
                    0.1314, 0.1354, 0.1302, 0])  # [no dim]
Y_REP_FOR_W = np.array([625, 750, 1000, 1250, 1500, 1750,
                        2000, 2250, 2500, 2750, 3000, 3250, 3500,
                        3750, 4000, 4250, 4500, 4750, 5000])  # [mm]
Y_MID_REP_FOR_W=np.array([687.5,875,1125,1375,1625,1875,2125,2375,2625,2875,3125,3375,
3625,3825,4125,4375,4625,4875])

RIBS_ARRAY=[625,1000,1500,2000,2500,3000,3500,4000,4500,5000]

Y_DISTANCE_FOR_W=np.array([Y_REP_FOR_W[i+1]-Y_REP_FOR_W[i] for i in range(0,Y_REP_FOR_W.shape[0]-1)])
W_REP = 9.8 * np.array([15, 12, 11, 7, 6, 5, 4, 4, 3,
                        4, 4, 3, 3, 3, 2, 2, 2, 1])#[N]

RHO_REP=W_REP/Y_DISTANCE_FOR_W



CHOORD_ARRAY=[2130,1070]
STA_FOR_CHOORD=[0,5000]

def getC_la(y):
    return np.interp(y, Y_REP_FOR_C, C_LA_REP)

def getC_lb(y):
    return np.interp(y,Y_REP_FOR_C,C_LB_REP)

def getC_d(y):
    return np.interp(y,Y_REP_FOR_C,C_D_REP)

def getRHO(y):
    return np.interp(y,Y_MID_REP_FOR_W,RHO_REP)

def getCl(y):
    return C_L*getC_la(y)+getC_lb(y)

def getCZ(y):
    return getCl(y)*np.cos(ALPHA_F)+getC_d(y)*np.sin(ALPHA_F)

def getChoord(y):
    return np.interp(y,STA_FOR_CHOORD,CHOORD_ARRAY)

def getCCzIntegral(y):
    step=1
    ans=sum([getChoord(y+step*i)*getCZ(y+step*i)*step for i in np.arange(np.floor((HARF_SPAN-y)/step))])
    return ans/1000/1000

def getSa(y):
    #ETA=44100/getCCzIntegral(0)
    ETA=4038
    return ETA*getCCzIntegral(y)

def getS1(y):
    step=100
    ans =N_Z*np.sum([getRHO(y+step*i)*step for i in np.arange (np.floor((HARF_SPAN-y)/step))])
    return ans

def getS(y):
    return getSa(y)-getS1(y)

def getM(y):
    step=100
    ans =np.sum([getS(y+step*i)*step for i in np.arange (np.floor((HARF_SPAN-y)/step))])
    return ans/1000

#前桁高さ
def getHf(y):
    HF_ARRAY=[320,130]
    Y_ARRAY_FOR_HF=[625,5000]
    return np.interp(y,Y_ARRAY_FOR_HF,HF_ARRAY)


In [12]:
M=[]
S=[]
for i in RIBS_ARRAY:
    M.append(getM(i))
    S.append(getS(i))
print("荷重分布")

print("y",",","S",",","M")
for i in range(len(RIBS_ARRAY)):
    print(RIBS_ARRAY[i],",",S[i],",",M[i])

荷重分布
y , S , M
625 , 31543.0320609 , 63119.1056958
1000 , 29266.4497764 , 51473.5421341
1500 , 24835.6275955 , 37706.2992465
2000 , 20171.006504 , 26220.7713898
2500 , 15628.9026554 , 17056.5823442
3000 , 11412.4525814 , 10104.7421366
3500 , 7668.18640412 , 5162.33941465
4000 , 4363.83311803 , 2007.92836779
4500 , 1613.47338848 , 405.885896007
5000 , 0.0 , 0.0


In [13]:
M=np.array(M)
S=np.array(S)

In [35]:
RIBS_ARRAY=np.array(RIBS_ARRAY)

In [36]:
df_SM=pd.DataFrame()

In [37]:
df_SM['y']=RIBS_ARRAY
df_SM['M']=M
df_SM['S']=S

In [38]:
df_SM

,y,M,S
0,625,63119.105696,31543.032061
1,1000,51473.542134,29266.449776
2,1500,37706.299247,24835.627595
3,2000,26220.771390,20171.006504
4,2500,17056.582344,15628.902655
5,3000,10104.742137,11412.452581
6,3500,5162.339415,7668.186404
7,4000,2007.928368,4363.833118
8,4500,405.885896,1613.473388
9,5000,0.000000,0.000000


In [45]:
df_frange=df_SM.copy()

In [47]:
df_frange['M_ULT']=1.5*M
df_frange['M_f']=0.8*1.5*M
df_frange['S_ULT']=1.5*S
df_frange['S_f']=0.8*1.5*S



In [48]:
df_frange

,y,M,S,M_ULT,M_f,S_ULT,S_f
0,625,63119.105696,31543.032061,94678.658544,75742.926835,47314.548091,37851.638473
1,1000,51473.542134,29266.449776,77210.313201,61768.250561,43899.674665,35119.739732
2,1500,37706.299247,24835.627595,56559.448870,45247.559096,37253.441393,29802.753115
3,2000,26220.771390,20171.006504,39331.157085,31464.925668,30256.509756,24205.207805
4,2500,17056.582344,15628.902655,25584.873516,20467.898813,23443.353983,18754.683187
5,3000,10104.742137,11412.452581,15157.113205,12125.690564,17118.678872,13694.943098
6,3500,5162.339415,7668.186404,7743.509122,6194.807298,11502.279606,9201.823685
7,4000,2007.928368,4363.833118,3011.892552,2409.514041,6545.749677,5236.599742
8,4500,405.885896,1613.473388,608.828844,487.063075,2420.210083,1936.168066
9,5000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [60]:
getHf(1500)

282.0

In [61]:
Hf=[]
for i in RIBS_ARRAY:
    Hf.append(getHf(i))

In [62]:
df_frange['Hf']=Hf

In [63]:
df_frange

,y,M,S,M_ULT,M_f,S_ULT,S_f,Hf
0,625,63119.105696,31543.032061,94678.658544,75742.926835,47314.548091,37851.638473,320.000000
1,1000,51473.542134,29266.449776,77210.313201,61768.250561,43899.674665,35119.739732,303.714286
2,1500,37706.299247,24835.627595,56559.448870,45247.559096,37253.441393,29802.753115,282.000000
3,2000,26220.771390,20171.006504,39331.157085,31464.925668,30256.509756,24205.207805,260.285714
4,2500,17056.582344,15628.902655,25584.873516,20467.898813,23443.353983,18754.683187,238.571429
5,3000,10104.742137,11412.452581,15157.113205,12125.690564,17118.678872,13694.943098,216.857143
6,3500,5162.339415,7668.186404,7743.509122,6194.807298,11502.279606,9201.823685,195.142857
7,4000,2007.928368,4363.833118,3011.892552,2409.514041,6545.749677,5236.599742,173.428571
8,4500,405.885896,1613.473388,608.828844,487.063075,2420.210083,1936.168066,151.714286
9,5000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,130.000000


In [64]:
def calcArea( b2,b1,t):
    area=b2*t+b1*t-t**2/4
    return area

In [65]:
def getCenterOfFrange(b2,b1,t):
    residure=calcArea(b2,b1,t)-b1*t
    height=residure/t
    return height

In [66]:
def getHe(hf,b2,b1,t):
    he=hf-getCenterOfFrange(b2,b1,t)